## SVARpy: SVAR-GMM Efficiency Gains
 
This file allows to replicate the estimations in .....
 

In [ ]:
# Install required packages

# If required pip install SVARpy package
if True:
    !pip install pathos 
    !pip install SVARpy  

import numpy as np 
import SVAR as SVAR
import SVAR.MoG as MoG
np.random.seed(0)

## Simulate the SVAR

Simulate reduced form shocks $u_t$ from the SVAR

$$
u_t = B_0 ɛ_t
$$

 with $n$ variables, $T$ observations, and structural shocks $ɛ_t$ generated from a mixture of normal distribution.

In [ ]:
n = 5  # Number of variables
T = 500  # Number of observations
# Specitfy B0
B0 = np.eye(n)

# shocks
mu1, sigma1 = (-0.2, np.power(0.7, 2))
mu2, sigma2 = (0.7501187648456057, np.power(1.4832737225521377, 2))
lamb = 0.7895
Omega = np.array([[mu1, sigma1], [mu2, sigma2]])

# Specitfy B_true
B_true = np.array([[1, 0  , 0  , 0, 0],
                    [0.5, 1, 0 , 0, 0],
                    [0.5, 0.5, 1, 0, 0],
                    [0.5, 0.5,  0.5  , 1, 0],
                    [0.5, 0.5,  0.5  , 0.5 , 1 ]])

V = np.linalg.cholesky(np.matmul(B_true, np.transpose(B_true)))
O = np.matmul(np.linalg.inv(V), B_true)
b_true = SVAR.SVARutil.get_Skewsym(O)
B_true = B_true * 10

# Draw structural shocks
eps = np.empty([T, n])
omega = np.zeros([n, 6])
for i in range(n):
        eps[:, i] = MoG.MoG_rnd(Omega, lamb, T)
        momentstmp = SVAR.MoG.MoG_Moments(Omega, lamb)[1:]
        for i in range(n):
            omega[i, :] = momentstmp

# Generate reduced form shocks
u = np.matmul(B_true, np.transpose(eps))
u = np.transpose(u)

 

array([1., 0., 0., 1.])

## Estimator: Cholesky

.....

In [ ]:
estimator = 'GMM'
estimator_name = 'Cholesky'
prepOptions = dict()
prepOptions['printOutput'] = True
prepOptions['bstartopt'] = 'Rec'
prepOptions['n_rec'] = n
prepOptions['addThirdMoments'] = False
prepOptions['addFourthMoments'] = False
prepOptions['moments_MeanIndep'] = False
prepOptions['moments_blocks'] = False
prepOptions['onlybivariate'] = False
prepOptions['Wpara'] = 'Uncorrelated'
prepOptions['Avarparametric'] = 'Uncorrelated'
prepOptions['Wstartopt'] = 'I'
prepOptions['S_func'] = False
prepOptions['kstep'] = 1 
GMM_out = SVAR.SVARest(u, estimator=estimator, prepOptions=prepOptions)


Estimator: GMM (steps=2)
Estimator Wopt/Avar: Uncorrelated/Uncorrelated
| SVAR            | Moments      | Tests             | E[e_1^m] / ... / E[e_n^m]     |
|-----------------|--------------|-------------------|-------------------------------|
| T=250           | #second: 3   | WaldRec=0.95      | m=2: 1.0 / 0.99               |
| n=2             | #third: 2    | WaldRec-pval=0.33 | m=3: -0.08 / 0.11             |
| #restrictions:0 | #fourth: 3   | J=4.37            | m=4: 1.85 / 1.7               |
| #unknowns:4     | ->loss: 0.02 | J-pval=0.36       |                               |
 
|                  |       B(:,1)       |      B(:,2)       |
|------------------|--------------------|-------------------|
|      B(1,:)      |        0.98        |       0.03        |
| (avar/wald/pval) | (0.21/1127.01/0.0) | (0.31/0.95/0.33)  |
|                  |                    |                   |
|      B(2,:)      |       -0.02        |       1.05        |
| (avar/wald/pval) |  (0.22/0.69

## Estimator: SVAR-GMM all moments

.....

In [ ]:

estimator = 'GMM'
estimator_name = 'GMM'
prepOptions = dict()
prepOptions['printOutput'] = True
prepOptions['bstartopt'] = 'Rec'
prepOptions['n_rec'] = n
prepOptions['addThirdMoments'] = True
prepOptions['addFourthMoments'] = True
prepOptions['moments_MeanIndep'] = False
prepOptions['moments_blocks'] = False
prepOptions['onlybivariate'] = False
prepOptions['Wpara'] = 'Uncorrelated'
prepOptions['Avarparametric'] = 'Uncorrelated'
prepOptions['Wstartopt'] = 'I'
prepOptions['S_func'] = False
prepOptions['kstep'] = 2 
GMM_out = SVAR.SVARest(u, estimator=estimator, prepOptions=prepOptions)


## Estimator: SVAR-GMM only non-redundant moments

.....

In [ ]:

estimator = 'GMM'
estimator_name = 'GMM-Biv'
prepOptions = dict()
prepOptions['printOutput'] = True
prepOptions['bstartopt'] = 'Rec'
prepOptions['n_rec'] = n
prepOptions['addThirdMoments'] = True
prepOptions['addFourthMoments'] = True
prepOptions['moments_MeanIndep'] = False
prepOptions['moments_blocks'] = False
prepOptions['onlybivariate'] = True
prepOptions['Wpara'] = 'Uncorrelated'
prepOptions['Avarparametric'] = 'Uncorrelated'
prepOptions['Wstartopt'] = 'I'
prepOptions['S_func'] = False
prepOptions['kstep'] = 2 
GMM_out = SVAR.SVARest(u, estimator=estimator, prepOptions=prepOptions)

## References

Keweloh, Sascha Alexander. "A generalized method of moments estimator for structural vector autoregressions based on higher moments." Journal of Business & Economic Statistics 39.3 (2021): 772-782.

Keweloh, Sascha Alexander. "A feasible approach to incorporate information in higher moments in structural vector autoregressions." (2021b).